In [ ]:
# Import Libraries
import os
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import csv
import time
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 

In [10]:
# Authentication Keys
# https://developer.twitter.com/en/portal/projects/1268606526274580480/apps/18064440/keys

# Think of these as the user name and password that represents
# your Twitter developer app when making API requests. 
consumerKey = '9gjPeGkMZj2935QAVEGEFqZG2'
consumerSecret = 'jSJbncxF8OifGepXcoR5uadfvhlVxXgoO9P5zDtMzNQidimRYx'

# User-specific credentials used to authenticate OAuth 1.0a API requests. 
# They specify the Twitter account the request is made on behalf of.
accessToken = '1130514658673213441-4GKVmzpYX67W2HlNiccXVIR3e0kzi7'
accessTokenSecret = 'HBN0T2QDg2nOcFOlZP0GyS9MwYGkjb3gtsrE7ebxKnJn8'

In [12]:
# Create the authentication object
auth = tweepy.OAuthHandler(consumerKey, consumerSecret) 
    
# Set the access token and access token secret
auth.set_access_token(accessToken, accessTokenSecret) 

# Creating the API object while passing in auth information
api = tweepy.API(auth, wait_on_rate_limit = True)

In [ ]:
# Constants 
MAX_REQUESTS = 180
TIME_WINDOW = 900  # 15 minutes = 900 seconds

In [27]:
# Define the search term and the date_since date as variables
search_words = '#CRA'
start_date = '2020-05-01'

# Collect tweets using Cursor method
# http://docs.tweepy.org/en/v3.5.0/cursor_tutorial.html
tweets = tweepy.Cursor(api.search,
              q=search_words,
              lang='en',
              since=start_date).items(5) 

# Iterate and print tweets
for tweet in tweets:
    print(tweet.text)

RT @RCAofficiel: ⚽#GoalofTheDay. The amazing goal scored by Saifeddine Alami last year against #CRA (Day 1)🚀

#DimaRaja https://t.co/0KxfqF…
RT @Sarah_Colero: This article is from 2018 and is still true to this day. It is the reason why 60% of Canada's disabled population are goi…
@CQualtro The #CRA needs to start working on the paper tax returns that they have been ignoring since March. Need t… https://t.co/yy5fZo4xVw
RT @Sarah_Colero: This article is from 2018 and is still true to this day. It is the reason why 60% of Canada's disabled population are goi…
RT @Sarah_Colero: This article is from 2018 and is still true to this day. It is the reason why 60% of Canada's disabled population are goi…


In [28]:
def buildTestSet(search_keyword):
    try:
        tweets_fetched = api.GetSearch(search_keyword, count = 100)
        
        print("Fetched " + str(len(tweets_fetched)) + " tweets for the term " + search_keyword)
        
        return [{"text":status.text, "label":None} for status in tweets_fetched]
    except:
        print("Unfortunately, something went wrong..")
        return None

In [ ]:
# Get search term input from user 
search_term = input("Enter a search keyword:")
testDataSet = buildTestSet(search_term)

# Print the first 5 tweets from data set
print(testDataSet[0:4])

In [ ]:
def buidTrainingSet(corpusFile, tweetDataFile):    
    corpus = []    
    with open(corpusFile,'rb') as csvfile:
        lineReader = csv.reader(csvfile,delimiter=',', quotechar="\"")
        for row in lineReader:
            corpus.append({"tweet_id":row[2], "label":row[1], "topic":row[0]})
            
    rate_limit = 180
    sleep_time = 900/180
    
    trainingDataSet = []
    
    for tweet in corpus:
        try:
            status = api.GetStatus(tweet["tweet_id"])
            print("Tweet fetched" + status.text)
            tweet["text"] = status.text
            trainingDataSet.append(tweet)
            time.sleep(sleep_time) 
        except: 
            continue
    # now we write them to the empty CSV file
    with open(tweetDataFile,'wb') as csvfile:
        linewriter = csv.writer(csvfile,delimiter=',',quotechar="\"")
        for tweet in trainingDataSet:
            try:
                linewriter.writerow([tweet["tweet_id"], tweet["text"], tweet["label"], tweet["topic"]])
            except Exception as e:
                print(e)
    return trainingDataSet

In [ ]:
corpusFile = "./corpus.csv"
tweetDataFile = "./tweetDataFile.csv"

trainingData = buildTrainingSet(corpusFile, tweetDataFile)

In [ ]:
class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        return [word for word in tweet if word not in self._stopwords]
    
tweetProcessor = PreProcessTweets()
preprocessedTrainingSet = tweetProcessor.processTweets(trainingData)
preprocessedTestSet = tweetProcessor.processTweets(testDataSet)

In [ ]:
def buildVocabulary(preprocessedTrainingData):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingData:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

In [ ]:
def extract_features(tweet):
    tweet_words=set(tweet)
    features={}
    for word in word_features:
        features['contains(%s)' % word]=(word in tweet_words)
    return features 

In [ ]:
# Now we can extract the features and train the classifier 
word_features = buildVocabulary(preprocessedTrainingSet)
trainingFeatures=nltk.classify.apply_features(extract_features,preprocessedTrainingSet)

In [ ]:
NBayesClassifier=nltk.NaiveBayesClassifier.train(trainingFeatures)

In [ ]:
NBResultLabels = [NBayesClassifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

In [ ]:
# get the majority vote
if NBResultLabels.count('positive') > NBResultLabels.count('negative'):
    print("Overall Positive Sentiment")
    print("Positive Sentiment Percentage = " + str(100*NBResultLabels.count('positive')/len(NBResultLabels)) + "%")
else: 
    print("Overall Negative Sentiment")
    print("Negative Sentiment Percentage = " + str(100*NBResultLabels.count('negative')/len(NBResultLabels)) + "%")

In [ ]:
"""
***Resources***

Authentication and Extracting tweets
https://www.earthdatascience.org/courses/use-data-open-source-python/intro-to-apis/twitter-data-in-python/

Cursor method 
http://docs.tweepy.org/en/v3.5.0/cursor_tutorial.html

YouTube Tutorial
https://www.youtube.com/watch?v=ujId4ipkBio
https://medium.com/better-programming/twitter-sentiment-analysis-15d8892c0082


Maybe for the ML stuff - has all the steps
https://towardsdatascience.com/creating-the-twitter-sentiment-analysis-program-in-python-with-naive-bayes-classification-672e5589a7ed

"""